# Conditional variational autoencoder (using the VAE class)

# SETUP

In [0]:
!python --version

Python 3.6.7


In [0]:
%%bash
git clone https://github.com/masa-su/pixyz.git

Cloning into 'pixyz'...


In [0]:
!pip --version


pip 19.0.2 from /usr/local/lib/python3.6/dist-packages/pip (python 3.6)


In [0]:
# !apt-get update && apt-get install ffmpeg

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch
!pip install tensorboardX
!pip install -e pixyz
torch.cuda.is_available()

# CVAE

generative process

3dimention data

position(x,y,θ)

c=f(ut,s(t-1))

st~p(st|ut, s(t-1))=N(c , sigma)

ot=N(h(st),sigma)

hは計測モデル，
ランドマークは(xc,yc,c)で構成，
hにより相対位置に変換される，
具体的には(距離と角度)


それぞれの分散共分散行列には対角行列を使用（制御がめんどそうだから

In [0]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
from tensorboardX import SummaryWriter
import numpy as np

from tqdm import tqdm

batch_size = 128
epochs =200#10
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [0]:
root = '../data'
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambd=lambda o: o.view(-1))])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

# train_loader = torch.utils.data.DataLoader(
#     datasets.MNIST(root=root, train=True, transform=transform, download=True),
#     shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(
#     datasets.MNIST(root=root, train=False, transform=transform),
#     shuffle=False, **kwargs)

In [0]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt
from matplotlib import rc
from IPython.display import HTML

In [0]:
from pixyz.distributions import Normal, Bernoulli
from pixyz.losses import KullbackLeibler
from pixyz.models import VAE

ModuleNotFoundError: ignored

In [0]:
#function for generate data
import torch.distributions as dist

def unif_landgenerator(sample_num):
  #range is [-180:180]
  sample_axis=torch.FloatTensor(sample_num,2).uniform_(0,100)
  sample_theta=torch.FloatTensor(sample_num).uniform_(-np.pi,np.pi)
  landmark=torch.cat([sample_axis,sample_theta.view(sample_num,1)],dim=1)
  return landmark

def get_h(st,lmap):
  #まずstから各lcまでの距離を計算して行列にいれとく
#   dis=torch.Tensor(len(lmap),len(st))
  disangle={}
  dis=torch.stack([torch.sqrt((st[:,0]-lmap[i][0])**2+(st[:,1]-lmap[i][1])**2)\
                   for i in range(len(lmap))],dim=0)
  angle=torch.stack([torch.atan2((lmap[i][0]-st[:,0]),(lmap[i][1]-st[:,1]))-st[:,2]\
                     for i in range(len(lmap))],dim=0)
  disangle.update({"dis":dis,"angle":angle})
  return disangle

#st*lmap*2  
def get_ot(st,lmap,r):#(robot_pos, map(landmarks), measurement_range)
  print(len(st),len(lmap))
  disangle=get_h(st,lmap)
  ot=torch.ones(len(st),len(lmap),2)*100.0
  for i in range(len(st)):
    for j in range(len(lmap)):
      if disangle["dis"][j][i]<=r:
        ot[i][j][0]=disangle["dis"][j][i]
        ot[i][j][1]=disangle["angle"][j][i]
  return ot#[sample_num*lmap_num*2[dis,angle]]
      
def inverse_ot_fast(st,ot,r):#inverse_otの処理を高速化した
  lnx=[]
  lny=[]
  bias=10.0
  for j in range(len(ot[0])):#landmark数
      lx=st[:,0]+torch.sin(ot[:,j,1]/bias+st[:,2]/bias)*ot[:,j,0]
      ly=st[:,1]+torch.cos(ot[:,j,1]/bias+st[:,2]/bias)*ot[:,j,0]
      lnx.append(lx)
      lny.append(ly)
  lnx=torch.stack(lnx,dim=1)#30000*20
  lny=torch.stack(lny,dim=1)
  ot2=ot[:,:,0]
  lx=lnx[ot2<=r]
  ly=lny[ot2<=r]
  
  return torch.stack([lx,ly],dim=1)

#st(x,y,theta),ot(dis,angle),r(range)->landmark_axis(x,y)
def inverse_ot(st,ot,r):
  ln=[]
  for i in range(len(st)):#sample数
    for j in range(len(ot[0])):#landmark数
      if ot[i][j][0]<=r:
        lx=st[i][0]+torch.sin(ot[i][j][1]/10.0+st[i][2]/10.0)*ot[i][j][0]
        ly=st[i][1]+torch.cos(ot[i][j][1]/10.0+st[i][2]/10.0)*ot[i][j][0]
        ln.append(torch.stack([lx,ly],dim=0))
#     if len(ln)==0:return ln
  return torch.stack(ln,dim=0)
      
  
#   return lc#lcはstごとのランドマーク行列
# def transform_randmark(st,lc,lmap):
#   st[:,0]

def origin_datagenerator(sample_num,dsigma,axis):
  data_cso={}
  
  ones=torch.ones(int(sample_num/3))
  
  sample_ca = torch.stack([ones*axis[0][0],ones*axis[0][1],ones*axis[0][2]],1)
  sample_cb = torch.stack([ones*axis[1][0],ones*axis[1][1],ones*axis[1][2]],1)
  sample_cc = torch.stack([ones*axis[2][0],ones*axis[2][1],ones*axis[2][2]],1)
  data_cso.update({"c":torch.cat([sample_ca,sample_cb,sample_cc],dim=0)})
  data_cso.update({"s":dist.Normal(data_cso["c"],dsigma).sample()})
  data_cso.update({"o":dist.Normal(data_cso["s"],dsigma).sample()})
#   data_cso.update([ ("c",torch.cat([sample_ca,sample_cb,sample_cc],dim=0)), ("s",dist.Normal(data_cso["c"],dsigma).sample()) ])
  
  return data_cso
  
#dataset sampling from Uniform[0:100]
def unique_datagenerator(sample_num,dsigma):
  data_cso={}
  
  sample_c1=torch.rand(sample_num)*100#一様分布からの場合
  sample_c2=torch.rand(sample_num)*100
  sample_theta=torch.FloatTensor(sample_num).uniform_(-np.pi,np.pi)
  data_cso.update({"c":torch.cat([sample_c1.view(sample_num,1),sample_c2.view(sample_num,1),sample_theta.view(sample_num,1)],dim=1)})
  data_cso.update({"s":dist.Normal(data_cso["c"],dsigma).sample()})
  return data_cso

In [0]:
sample_num=6000
landmark_num=20
range_of_measure=200
dsigma=torch.Tensor([0.3,0.3,0.3]*sample_num).view(sample_num,3)
axis=[[30,9,0.5],[40,90,-1.4],[72,60,-3.0]]

max_data=100.0
min_data=0.0

In [0]:
# #data generate
# lmap=unif_landgenerator(landmark_num)
# data=unique_datagenerator(sample_num,dsigma)

In [0]:
#data generate

#generate landmark,condition,true_st
lmap=unif_landgenerator(landmark_num)
data=origin_datagenerator(sample_num,dsigma,axis)
# test_data=origin_datagenerator(sample_num,dsigma,axis)
ot=get_ot(data["s"],lmap,range_of_measure)
print(ot)

#角度のdataを10倍で合わせる
ot[:,:,1]=torch.atan2(torch.sin(ot[:,:,1]),torch.cos(ot[:,:,1]))*10
data["c"][:,2]=data["c"][:,2]*10
data["s"][:,2]=torch.atan2(torch.sin(data["s"][:,2]),torch.cos(data["s"][:,2]))*10


# ot=ot.view(sample_num,landmark_num*2)
data.update({"o":ot.view(sample_num,landmark_num*2)})
# print(ot.size())
print(data["c"].size())
train = torch.utils.data.TensorDataset(data["o"], data["c"])
train_loader = torch.utils.data.DataLoader(train, shuffle=False,**kwargs)
# test = torch.utils.data.TensorDataset(test_data["o"], test_data["c"])
# test_loader = torch.utils.data.DataLoader(test, shuffle=False,**kwargs)

In [0]:
# #obserbation
# # print(get_h(data["c"],lmap))
# iot=inverse_ot_fast(data["s"],ot,range_of_measure)
# # print(iot.size())
# plt.scatter(iot[:,0],iot[:,1],color='blue',marker='o',edgecolors="red")
plt.scatter(lmap[:,0],lmap[:,1],color='red',marker='x',edgecolors="red")
# plt.scatter(data["c"][:,0],data["c"][:,1],color='yellow',marker='x',edgecolors="red")
plt.xlim(min_data,max_data)
plt.ylim(min_data,max_data)
plt.xlabel('$sx$', fontsize=16)
plt.ylabel('$sy$', fontsize=16)
plt.title("Simulated dataset")
plt.show()

In [0]:
o_dim = landmark_num*2
c_dim = 3
s_dim = 3
hidden=(landmark_num*2+3)


# inference model q(z|x,y)
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["o","c"], var=["s"], name="q")

        self.fc1 = nn.Linear(o_dim+c_dim, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, hidden)
        self.fc31 = nn.Linear(hidden, s_dim)
        self.fc32 = nn.Linear(hidden, s_dim)

    def forward(self, o, c):
        h = F.relu(self.fc1( torch.cat([o,c],1) ))
        h = F.relu(self.fc2(h))
        h = F.relu(self.fc3(h))
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}

    
# generative model p(x|z,y)   
class Generator2(Normal):
    def __init__(self):
        super().__init__(cond_var=["s"], var=["o"], name="p")

        self.fc1 = nn.Linear(s_dim, hidden)
        self.fc2 = nn.Linear(hidden, hidden)
        self.fc25 = nn.Linear(hidden, hidden)
        self.fc3 = nn.Linear(hidden, o_dim)

    def forward(self, s):
#         print(s)
#         print(s.shape)
        h = F.relu(self.fc1(s))
        h = F.relu(self.fc2(h))
        h = F.relu(self.fc25(h))
        return {"loc": self.fc3(h),"scale":(torch.ones(o_dim)*0.3).to(device)}

# prior = Normal(loc=loc, scale=scale, cond_var=["c"],var=["s"], dim=s_dim, name="p_prior")

In [0]:
# prior model p(z|c)
class prior_set(Normal):
    def forward(self, c):
        return{"loc":c, "scale":torch.tensor([0.3,0.3,0.3]).to(device)}
      
prior = prior_set(cond_var=["c"],var=["s"], dim=s_dim, name="p_prior")
print(prior)

In [0]:
p = Generator2()
q = Inference()

p.to(device)
q.to(device)

print(p)
print(q)

In [0]:
kl = KullbackLeibler(q, prior)
print(kl)

In [0]:
model = VAE(q, p, regularizer=kl, optimizer=optim.Adam, optimizer_params={"lr":1e-3})
print(model)

In [0]:
def train(epoch):
    train_loss = 0
    for o,c in tqdm(train_loader):
        o = o.to(device)
        c = c.to(device)
#         print(o.size())
#         print(c.size())
#         print(o)
        loss = model.train({"o": o, "c": c})
        train_loss += loss
        
#     print(kl.gauss_gauss_kl(, scale1, loc2, scale2, dim=None))
    train_loss = train_loss * train_loader.batch_size / len(data["c"])
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [0]:
# print(train_loader.batch_size)
# print(len(train_loader))
# print(dc)

In [0]:
def test(epoch):
    test_loss = 0
    for o, c in test_loader:
        o = o.to(device)
        c = c.to(device)
        loss = model.test({"o": o, "c": c})
        test_loss += loss
        

    test_loss = test_loss * test_loader.batch_size / len(test_data["c"])
    print('Test loss: {:.4f}'.format(test_loss))
    return test_loss

In [0]:
# model.test({"o":do[0].unsqueeze(0).to(device),"c":dc[0].unsqueeze(0).to(device)})
# q.sample("o":do[0],"c":dc[0], return_all=False)

In [0]:
def plot_reconstrunction(o, c):
    with torch.no_grad():
        s = q.sample({"o": o, "c": c}, return_all=False)
        s.update({"c": c})
        recon_batch = p.sample_mean(s).view(-1, 1, 28, 28)
    
        recon = torch.cat([o.view(-1, 1, 28, 28), recon_batch]).cpu()
        return recon
    
def plot_image_from_latent(s, c):
    with torch.no_grad():
        sample = p.sample_mean({"s": s, "c": c}).view(-1, 1, 28, 28).cpu()
        return sample
    
def plot_reconstrunction_changing_y(o, c):
    c_change = torch.eye(10)[range(7)].to(device)
    batch_dummy = torch.ones(o.size(0))[:, None].to(device)    
    recon_all = []
    
    with torch.no_grad():
        for _c in c_change:
            s = q.sample({"o": o, "c": c}, return_all=False)
            s.update({"c": batch_dummy * _c[None,:]})
            recon_batch = p.sample_mean(s).view(-1, 1, 28, 28)
            recon_all.append(recon_batch)
    
        recon_changing_c = torch.cat(recon_all)
        recon_changing_c = torch.cat([o.view(-1, 1, 28, 28), recon_changing_c]).cpu()
        return recon_changing_c

In [0]:
#prepare for plot
history = {"train_loss":[],"test_loss":[]}

ims=[]#animation variable
# sample_s=dist.Normal(dc,dsigma).sample()
# sample_o=dist.Normal(sample_s,dsigma).sample()

grid_inter=10.0

plt.xlim([min_data,max_data])
plt.ylim([min_data,max_data])

plt.xlabel('$sx$', fontsize=16)
plt.ylabel('$sy$', fontsize=16)
plt.title("Simulated dataset")

# plt.xticks(np.arange(-2,6,1.0))

In [0]:
writer = SummaryWriter()

plot_number = 1

s_sample = 0.5 * torch.randn(64, s_dim).to(device)
c_sample = torch.eye(10)[[plot_number]*64].to(device)

# _o, _c = iter(test_loader).next()
# _o = _o.to(device)
# _c = torch.eye(10)[_c].to(device)


for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
#     test_loss = test(epoch)
    
#     recon = plot_reconstrunction(_o[:8], _c[:8])
#     sample = plot_image_from_latent(s_sample, c_sample)
#     recon_changing_c = plot_reconstrunction_changing_c(_o[:8], _c[:8])

    writer.add_scalar('train_loss', train_loss.item(), epoch)
#     writer.add_scalar('test_loss', test_loss.item(), epoch)
    
    #for plot
    history["train_loss"].append(train_loss.item())
#     history["test_loss"].append(test_loss.item())

    _s = q.sample({"o": data["o"].to(device), "c": data["c"].to(device)}, return_all=False)
    plt.subplot(1, 2, 1)
    tru = plt.scatter(data["s"][:,0],data["s"][:,1],color='blue',marker='o',edgecolors="blue")
    plt.xticks(np.arange(min_data,max_data+20,grid_inter))
    plt.yticks(np.arange(min_data,max_data+20,grid_inter))
    plt.ylabel('$sy$', fontsize=16)
    plt.xlabel('$sx$', fontsize=16)
    
    plt.subplot(1, 2, 2)
    sam = plt.scatter(_s["s"][:,0].cpu(),_s["s"][:,1].cpu(),color='red',marker='x',edgecolors="red")
    plt.xticks(np.arange(min_data,max_data+20,grid_inter))
    plt.yticks(np.arange(min_data,max_data+20,grid_inter))
    plt.xlabel('$sx$', fontsize=16)
    ims.append([tru]+[sam])
    plt.show()
    

#     writer.add_image('Image_from_latent', sample, epoch)
#     writer.add_image('Image_reconstrunction', recon, epoch)
#     writer.add_image('Image_reconstrunction_change_c', recon_changing_c, epoch)
    
writer.close()

In [0]:
# #animation
# def iinit():    
#     return (ims[0])
# # animation function: this is called sequentially
# def animate(i):
#   return (ims[i])

# #plot gif
# fig = plt.figure()
# plt.legend(['True','Result'], loc='upper left', fontsize=16)
# ani = animation.ArtistAnimation(fig, ims)
# anim = animation.FuncAnimation(fig, animate, init_func=iinit, frames=epochs, interval=100, blit=True)
# plt.rcParams['animation.ffmpeg_path'] = '/usr/bin/ffmpeg' # For google colab
# HTML(ani.to_html5_video())
# rc('animation', html='jshtml')
# anim

In [0]:
#plot loss    
plt.ylabel('$loss$', fontsize=16)
plt.xlabel('$epoch$', fontsize=16)
plt.title("train_loss")
plt.plot(range(epochs), history["train_loss"])
plt.show()
# plt.title("test_loss")
# plt.plot(range(epochs), history["test_loss"])
# plt.show()

In [0]:
# sample_num=300

# dsigma=torch.Tensor([0.3,0.3]*sample_num).view(sample_num,2)
#sample dataset

# ca_axis=[-3,-2]
# cb_axis=[3,3]
# cc_axis=[7,5]

# ones=torch.ones(int(sample_num/3))

# #sample dataset
# #cを自分で定義した場合
# sample_ca = torch.stack([ones*ca_axis[0],ones*ca_axis[1]],0)
# sample_cb = torch.stack([ones*cb_axis[0],ones*cb_axis[1]],0)
# sample_cc = torch.stack([ones*cc_axis[0],ones*cc_axis[1]],0)
# sample_c=torch.cat([sample_ca,sample_cb,sample_cc],dim=1).view(sample_num,2)



# sample_c=torch.rand(sample_num)*100#一様分布からの場合

# sample_s=dist.Normal(sample_c,dsigma).sample()
# sample_o=dist.Normal(sample_s,dsigma).sample()

# _s = q.sample_mean({"o": sample_o.to(device), "c": sample_c.to(device)})
# data=origin_datagenerator(sample_num,dsigma,axis)
_s = q.sample({"o": data["o"].to(device), "c": data["c"].to(device)}, return_all=False)
_o= p.sample({"s": _s["s"].to(device)}, return_all=False)

In [0]:
# print(_s)
# print(_o["o"])
# print(_o["o"].view(sample_num,landmark_num,2))
# oot=ot.view(sample_num,landmark_num,2).to(device)
_oo=_o["o"].view(sample_num,landmark_num,2).cpu()
# print(_oo.size())
# print(_oo[0][0])
_oo_loss=0
ct=0
for i in range(sample_num):
  for j in range(landmark_num):
    if _oo[i][j][0]<=200:
      _oo_loss+=torch.abs(_oo[i][j]-ot[i][j])
      ct+=1
print(_oo_loss/ct)
# print(data["s"])

In [0]:


# _s = q.sample({"o": ot.to(device), "c": data["c"].to(device)}, return_all=False)
# _o= p.sample({"s": _s["s"].to(device)}, return_all=False)

In [0]:
sample_num=1
tamesi=unique_datagenerator(sample_num,torch.Tensor([0.3,0.3,0.3]*sample_num).view(sample_num,3))
tamesi["s"][:,2]=torch.atan2(torch.sin(tamesi["s"][:,2]),torch.cos(tamesi["s"][:,2]))
_oo= p.sample({"s": tamesi["s"].to(device)}, return_all=False)["o"].view(sample_num,landmark_num,2)
_oo_true=get_ot(tamesi["s"],lmap,range_of_measure)

np.set_printoptions(precision=5)
print(_oo)
print(_oo_true)
print(tamesi)

In [0]:
print(_s)
print(_o["o"])
print(data["s"])
print(_oo.size())
# _ioo=inverse_ot_fast(tamesi["s"].to(device),_oo,range_of_measure)
_ioo=inverse_ot_fast(tamesi["s"].to(device),_oo.to(device),range_of_measure)
sample_num=20000
iot=inverse_ot_fast(data["s"].to(device),ot.to(device),range_of_measure)


# plt.scatter(tamesi["s"][:,0],tamesi["s"][:,1],color='yellow',marker='o',edgecolors="yellow")
# plt.scatter(sample_o[:,0],sample_o[:,1],color='blue',marker='o',edgecolors="blue")

plt.scatter(_ioo[:,0].cpu(),_ioo[:,1].cpu(),color='blue',marker='x',edgecolors="red")
plt.scatter(iot[:,0].cpu(),iot[:,1].cpu(),color='red',marker='x',edgecolors="red")
plt.xlabel('$sx$', fontsize=16)
plt.ylabel('$sy$', fontsize=16)
plt.title("Simulated dataset")
plt.show()

In [0]:
loss=0
_ss=_s["s"].cpu()
for i in range(0,len(data["s"][i])-1,1):
    loss+=torch.abs(data["s"][i]-_ss[i])
print(loss/sample_num)

In [0]:
max_data=10#max_data
min_data=-4#min_data
grid_inter=1

plot_data=test_data

_s = q.sample({"o": plot_data["o"].to(device), "c": plot_data["c"].to(device)}, return_all=False)

#show sample points
plt.scatter(plot_data["s"][:,0],plot_data["s"][:,1],color='blue',marker='o',edgecolors="blue")
plt.scatter(_s["s"][:,0],_s["s"][:,1],color='red',marker='x',edgecolors="red")
plt.xlabel('$sx$', fontsize=16)
plt.ylabel('$sy$', fontsize=16)
plt.title("Simulated dataset")
plt.show()

plt.subplot(1, 2, 1)
plt.scatter(plot_data["s"][:,0],plot_data["s"][:,1],color='blue',marker='o',edgecolors="blue")
plt.xticks(np.arange(min_data-1,max_data+1,grid_inter))
plt.subplot(1,2,2)
plt.scatter(_s["s"][:,0],_s["s"][:,1],color='red',marker='x',edgecolors="red")
plt.xticks(np.arange(min_data-1,max_data+1,grid_inter))
plt.show()
